# Mounting drive

In [1]:
#To login to your googledrive account for file management
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#CD into the directory you want to be in and clone the repository in
% cd drive/My Drive/Colab Notebooks/WRI_Enivronmental_Conflicts/

/content/drive/My Drive/Colab Notebooks/WRI_Enivronmental_Conflicts


In [3]:
#!pip uninstall spacy 
#!pip uninstall neuralcoref
!pip install spacy==2.1.0
!pip install neuralcoref
!python -m spacy download en

     |████████████████████████████████| 27.7MB 44.8MB/s 
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9
     |████████████████████████████████| 296kB 9.2MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


# Creating custom entity detection

**Loading in the file with prelabeled entities. Splitting into train and test**

In [4]:
import spacy
import neuralcoref
import random

100%|██████████| 40155833/40155833 [00:02<00:00, 19787185.31B/s]


In [0]:
import json
import random

#Converting JSON1 files to Spacy tuples format
def convert_doccano_to_spacy(filepath):
    with open(filepath, 'rb') as fp:
        data = fp.readlines()

    training_data = []
    for record in data:
        entities = []
        read_record = json.loads(record)
        text = read_record['text']
        entities_record = read_record['labels']

        for start, end, label in entities_record:
            entities.append((start, end, label))

        training_data.append((text, {"entities": entities}))
    return training_data

In [0]:
filepath = 'corefs_positive_second_batch.json1'
entities = convert_doccano_to_spacy(filepath)

In [89]:
#We remove all those articles that haven't been annotated from the training set and put them in the test set
test = []
train = []
for i in range(175):
  if (entities[i][1] == {'entities': []}):
    test.append(entities[i])
  else:
    train.append(entities[i]) 
print(len(train))
#Next we add some more news articles from the training set into the test set
r = random.randint(0,100)
for i in range(r,r+20):
  test.append(train[i])
  train.remove(train[i])

147


In [90]:
len(test)

48

In [0]:
#Restricting our news article to the first four paragraphs. 
def chop_news(article):
    paragraphs = article.splitlines()[:4]
    new_text = ' '.join(paragraphs)
    return new_text

# Training the Model

In [69]:
#Train a blank english model with our custom entities



TRAIN_DATA = train


def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

    nlp.add_pipe(nlp.create_pipe('sentencizer'))  #Adding sentencizer as a prerequisite to coref
    neuralcoref.add_to_pipe(nlp)  #Adding corefering in the pipeline
    
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA, 20)

# Save our trained Model
#modelfile = input("Enter your Model Name: ")
prdnlp.to_disk('Custom_NER_Model')



Starting iteration 0
{'ner': 4490.837859520542}
Starting iteration 1
{'ner': 4920.839953297986}
Starting iteration 2
{'ner': 3204.9036175011156}
Starting iteration 3
{'ner': 4216.418759342657}
Starting iteration 4
{'ner': 3446.6095027448728}
Starting iteration 5
{'ner': 4563.0620263103165}
Starting iteration 6
{'ner': 5001.291738912865}
Starting iteration 7
{'ner': 2789.93067522007}
Starting iteration 8
{'ner': 4249.560251776801}
Starting iteration 9
{'ner': 2745.8087251758893}
Starting iteration 10
{'ner': 3403.094466017772}
Starting iteration 11
{'ner': 2532.117717726096}
Starting iteration 12
{'ner': 3392.292258568213}
Starting iteration 13
{'ner': 2862.723560541181}
Starting iteration 14
{'ner': 3405.06930386159}
Starting iteration 15
{'ner': 3021.045569118008}
Starting iteration 16
{'ner': 3398.013676563408}
Starting iteration 17
{'ner': 2869.9961363740094}
Starting iteration 18
{'ner': 2624.7629153403204}
Starting iteration 19
{'ner': 3017.878466013696}


# Testing and diplaying

In [0]:
# Print the entities
import pandas as pd

def get_entities_list(text,doc):
  actor = []
  number = []
  action = []
  for ents in doc.ents:
    if ents.label_ == 'actor':
      actor.append(ents.text)
    elif ents.label_ == 'number':
      number.append(ents.text)
    else:
      action.append(ents.text) 
  entities = [actor, action, number]
  for e in entities:
    e = list(dict.fromkeys(e))    #removing duplicated
  len_table = max(len(actor), len(action), len(number))
  for e in entities:
    e += [''] * (len_table - len(e))
  #print('Actor', actor)
  #print('Action', len(action))
  #print('Number', number)   
   
  entities = ['Actor', 'Number', 'Action'] 
  data = [entities] + list(zip(actor, number, action))   
  for i, d in enumerate(data):
    line = '|'.join(str(x).ljust(50) for x in d)  #pretty printing
    print(line)
    if i == 0:
        print('-' * len(line))
  return None

#get_entities_list('asdfsdf',doc)  

In [91]:
#Test your text
from spacy import displacy
#test_text = 'Bharatiya Janata Party President Amit Shah on Sunday launched an attack on the Communist Party of India (Marxist) (CPI-M) in Tripura for spreading violence, saying that the former cannot stop them from rising in the Left-ruled state.\nAhead of the Assembly elections in Tripura, the BJP Chief, while addressing a public rally here, said, "I would like to tell the Manik government that the BJP will not cower down by violence. The more you spread the mud of violence, the better will be the Lotus blossom".\nHe added that more than seven lakh people, out of the 37 lakh population of Tripura, are registered in the unemployment list.\n"The health facilities here are inadequate. This is what was done here in the past 25 years by the ruling state government. But not the countdown has begun. The BJP will form the government here in March. Even if the corrupt goes underground, our party will dig the earth and bring them out," Shah asserted.\nHe further said that the Tripura government is not taking any action against the culprits of chit fund because the entire Cabinet of Manik government will be behind the bars.\nThe Assembly elections in the states will be held in March.\nBoth Nagaland and Tripura have been two of the few states where the BJP has not formed a government.\nIn the current Nagaland assembly, the Naga People\'s Front (NPF) hold the majority.\nMeanwhile, in Tripura Assembly, the Communist Party of India (Marxist) has been enjoying majority, while Chief Minister Manik Sarkar has been at the helm since 1998.'
#test_text = 'Turkey has captured the elder sister of the slain leader of the Islamic State group in north-western Syria, according to a senior Turkish official, who called the arrest an intelligence "gold mine". \n Little is known about the sister of Abu Bakr al-Baghdadi. The Turkish official said the 65-year-old known as Rasmiya Awad is suspected of being affiliated with the extremist group. He did not elaborate. \n Awad was captured in a raid Monday evening (local time) on a trailer container she was living in with her family near the town of Azaz in Aleppo province. \n The area is part of the region administered by Turkey after it carried out a military incursion to chase away IS militants and Kurdish fighters starting 2016. Allied Syrian groups manage the area known as the Euphrates Shield zone. \n The official said the sister was with her husband, daughter-in-law and five children. The adults are being interrogated, he said, speaking on condition of anonymity in line with government protocol.\n "This kind of thing is an intelligence gold mine. What she knows about (IS) can significantly expand our understanding of the group and help us catch more bad guys," the official said. \n Al-Baghdadi, an Iraqi from Samarra, was killed in a US raid in the nearby province of Idlib last month. The raid was a major blow to the group, which has lost territories it held in Syria and Iraq in a series of military defeats by the US-led coalition and Syrian and Iraqi allies.'
for i in range(0,19):
  test_text = chop_news(test[i][0])
  doc = prdnlp(test_text)
  get_entities_list(test_text,doc)
  #for ent in doc.ents:
  #  print(ent.text, ent.start_char, ent.end_char, ent.label_)
  options = {"ents": ["ACTOR", "ACTION", "NUMBER"],
           "colors": {"ACTOR": "red", "ACTION":"blue", "NUMBER":"green"}}
  displacy.render(doc, style="ent", options = options, jupyter = True )             

Actor                                             |Number                                            |Action                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------
financial industry                                |                                                  |                                                  
finance industry                                  |                                                  |                                                  
maintained                                        |                                                  |                                                  


Actor                                             |Number                                            |Action                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------
farmers                                           |                                                  |                                                  
farmers                                           |                                                  |                                                  
farmers                                           |                                                  |                                                  


Actor                                             |Number                                            |Action                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------
Both leaders                                      |                                                  |                                                  


Actor                                             |Number                                            |Action                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------
Odisha government                                 |                                                  |                                                  


Actor                                             |Number                                            |Action                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------
state government                                  |                                                  |                                                  


...

Actor                                             |Number                                            |Action                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------
state Congress president                          |                                                  |                                                  


Actor                                             |Number                                            |Action                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------
New Delhi                                         |                                                  |                                                  
Delhi Jal Board                                   |                                                  |                                                  
Delhi Jal Board                                   |                                                  |                                                  


Actor                                             |Number                                            |Action                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------
Supreme Court                                     |                                                  |been affected                                     
public                                            |                                                  |elevated road where the work is still on, however, bajri stock here will last only till January 15. With no solution in sight for the bajri problem, it is unlikely that work
JDC                                               |                                                  |                                                  
voters                                            |                                                  |          

Actor                                             |Number                                            |Action                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------
forest officials                                  |                                                  |attacked                                          
farmer                                            |                                                  |killed                                            
farmer                                            |                                                  |                                                  
tiger                                             |                                                  |                                                  
farmer                                            |                               

Actor                                             |Number                                            |Action                                            
--------------------------------------------------------------------------------------------------------------------------------------------------------
Centre                                            |                                                  |expected to set                                   
BADP                                              |                                                  |India                                             
state government                                  |                                                  |                                                  
China                                             |                                                  |                                                  
India                                             |                               

# Getting location and date entities